In [5]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import numpy as np

df = yf.Ticker("600887.SS").history(start='2020-08-01', end='2021-11-01')[['Open','High','Low', 'Close']]
# process data
df.ta.macd(close='close', fast=12, slow=26, append=True)
df.columns = [x.lower() for x in df.columns]
# when MACD crosses above the signal line (the histogram larger than 0 and becomes red), buy
buy_signal = ((df.macd_12_26_9.shift(1) < df.macds_12_26_9.shift(1)) & (df.macd_12_26_9 > df.macds_12_26_9)).astype(int).rename("buy_signal")
# when MACD crosses below the signal line (the histogram smaller than 0 and becomes green), sell.
sell_signal = ((df.macd_12_26_9.shift(1) > df.macds_12_26_9.shift(1)) & (df.macd_12_26_9 < df.macds_12_26_9)).astype(int).rename("sell_signal")
integrated_data = pd.concat([df, buy_signal, sell_signal], axis=1, join='inner')
test_data = integrated_data[integrated_data.index >= '2020-11-01']

initial_balance = 1000000
initial_shares = 0

cash_balance = initial_balance
share_holding = initial_shares
yesterday_flag = 0
trading_outlay = list()

trading_results = pd.DataFrame(columns=["date","signal","cash_balance", "share_holding", "asset"
                                        , "transaction_price", "transaction_cost", "trading_outlay", "reward" ])

for row in test_data.iterrows():

    reward = 0
    transaction_price = (row[1].high+row[1].low)/2
    transaction_cost = 0

    if yesterday_flag == 1: #buy
#         print('buy')
        # the logic here is to assume I pay the highest fees, how many hands can I afford
        max_hand = np.floor((cash_balance)/(transaction_price*1.00302)/100)
        transaction_shares = max_hand * 100
        
        transaction_amount = transaction_price*transaction_shares
        commision_charge = max(transaction_amount*0.003,5)
        transfer_fee = transaction_amount*0.00002
        transaction_cost = commision_charge+transfer_fee 
        
        cash_balance = cash_balance-transaction_amount-transaction_cost
        share_holding = share_holding+transaction_shares
        
        trading_outlay.append(transaction_cost+transaction_amount)
        
    elif yesterday_flag == -1: #sell
#         print('sell')
        transaction_shares = share_holding
        transaction_amount = transaction_price*transaction_shares
        commision_charge = max(transaction_amount*0.003,5)
        transfer_fee = transaction_amount*0.00002        
        stamp_tax = transaction_amount*0.001                
        transaction_cost = commision_charge+transfer_fee+stamp_tax
        cash_balance = cash_balance+transaction_amount-transaction_cost
        share_holding = 0
        reward = transaction_amount-transaction_cost-np.sum(trading_outlay)
        trading_outlay = list()

    
    asset = round(cash_balance+share_holding*transaction_price,0)
#     print(trading_outlay)
    trading_results = trading_results.append({"date":row[0],"signal":yesterday_flag,"cash_balance":round(cash_balance,2)
                                              , "share_holding":share_holding, "asset":asset
                                              ,"transaction_price":transaction_price, "transaction_cost":transaction_cost
                                              ,"trading_outlay": np.sum(trading_outlay),"reward":reward
                                             }, ignore_index=True)

    # observe today's trading signal and carry out tomorrow
    if row[1].buy_signal == 1:
        yesterday_flag = 1
    elif row[1].sell_signal == 1:
        yesterday_flag = -1
    else:
        yesterday_flag = 0
        
trading_results.to_excel("trading_results.xlsx")

In [6]:
# 0.3/100 + 0.2/1e4

In [16]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import numpy as np

# make a 2 x 1 figure
fig = make_subplots(rows=2, cols=1)

# add candlestick on the upper subplot
fig.append_trace(
    go.Candlestick(
        x=test_data.index,
        open=test_data.open,
        high=test_data.high,
        low=test_data.low,
        close=test_data.close,
        increasing_line_color='#ef476f',
        decreasing_line_color='#06d6a0',
        showlegend=False
    ), row=1,col=1
)

# plot macd on the lower subplot
fig.append_trace(
    go.Scatter(
        x=test_data.index,
        y=test_data.macd_12_26_9,
        line=dict(color='#118ab2', width=2),
        name='macd',
        legendgroup='2',
    ), row=2,col=1
)

# plot signal line on the lower subplot
fig.append_trace(
    go.Scatter(
        x=test_data.index,
        y=test_data.macds_12_26_9,
        line=dict(color='#ffd166',width=2),
        legendgroup='2',
        name='signal'
    ), row=2, col=1
)

# define histogram colors
colors = np.where(test_data.macdh_12_26_9<0, '#06d6a0', '#ef476f')

# add histogram on the lower subplot
fig.append_trace(
    go.Bar(
        x=test_data.index,
        y=test_data.macdh_12_26_9,
        name='histogram',
        marker_color=colors,
    ),row=2,col=1
)

# plot buying signal
buying_signal = trading_results[trading_results.signal == 1]

fig.append_trace(
    go.Scatter(
        x=buying_signal.date,
        y=buying_signal.transaction_price*0.96,
        marker = dict(
            color='#073b4c',
            size=10,
            line=dict(
                color='#118ab2',
                width=2
            ),
            symbol='triangle-up'
            
        ),
        mode = "markers+text",
        name = "buy",
        text = buying_signal.transaction_price,
        textposition="bottom center",
        texttemplate='%{text:$.2f}',
        
    ), row=1, col=1
)

# plot selling signal
sell_signal = trading_results[trading_results.signal == -1]

fig.append_trace(
    go.Scatter(
        x=sell_signal.date,
        y=sell_signal.transaction_price*1.04,
        marker = dict(
            color='#fb5607',
            size=10,
            line=dict(
                color='#ffbe0b',
                width=2
            ),
            symbol='triangle-down'
            
        ),
        mode = "markers+text",
        name = "sell",
        text = sell_signal.transaction_price,
        textposition="top center",
        texttemplate='%{text:$.2f}',
#         textsize = 1
    ), row=1, col=1
)


# make the figure prettier
layout = go.Layout(
    plot_bgcolor='#ecf8f8',
    font_family='Monospace',
    font_color='#073b4c',
    font_size=10,
    xaxis=dict(
        rangeslider=dict(visible=False)
    ),
    autosize=False,
    width=1000,
    height=400,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    )
)

# ignore Saturdays and Sundays
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=['sat', 'mon'])
    ]
)

# plot figure
fig.update_layout(layout)
fig.show()

In [8]:
fig.write_image("fig1.jpeg")

In [9]:
buying_signal

,date,signal,cash_balance,share_holding,asset,transaction_price,transaction_cost,trading_outlay,reward
24,2020-12-04,1,737.72,25800.0,996991.0,38.614480,3008.685844,9.992623e+05,0
74,2021-02-22,1,3522.05,25700.0,1190945.0,46.203226,3586.017219,1.191009e+06,0
96,2021-03-24,1,31.7,28800.0,1113257.0,38.653647,3361.939589,1.116587e+06,0
132,2021-05-19,1,2315.91,27900.0,1061766.0,37.973109,3199.538184,1.062649e+06,0
173,2021-07-16,1,642.52,29100.0,1029910.0,35.370001,3108.386414,1.032375e+06,0
185,2021-08-03,1,1482.34,26800.0,934122.0,34.799999,2816.572738,9.354566e+05,0
207,2021-09-02,1,2185.63,24400.0,883270.0,36.110001,2660.873725,8.837449e+05,0
222,2021-09-27,1,3329.33,22500.0,853829.0,37.799999,2568.509948,8.530685e+05,0
238,2021-10-26,1,3917.88,21100.0,858362.0,40.495001,2580.422443,8.570249e+05,0
